# Review: Pre-Activation ResNet with Identity Mapping — Over 1000 Layers Reached (Image Classification)

### Pre-Activation ResNet: Batch Norm and ReLU before Convolution

>* SH Tsang의 [Pre-Activation ResNet](https://towardsdatascience.com/resnet-with-identity-mapping-over-1000-layers-reached-image-classification-bb50a42af03e)을 번역한 글입니다.
>
>* [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027) 논문을 리뷰한 글입니다.

이번 장에서는 Microsoft가 발전시킨 Resnet에 대해 살펴보자. <strong>'Identity Mapping'</strong> 기술을 사용해서 에러의 증가없이도 1000 layer가 넘는 딥러닝 구조(deep learning architecture)을 구현할 수 있게 되었다. 

기존 Resnet에서는 101-layer를 1202-layer로 만들었을때, Resnet-1202이 수렴(converge)하더라도 에러율(error rate)이 6.43%에서 7.93%로 증가하는 모습을 보였다. 기존 논문에서는 해당 현상에 대해 자세한 설명이 없는 상태였다(open question).

아래 그림은 Identity Mapping을 사용한 Resnet의 결과를 보여주고 있다. 1001-layer를 쌓은 상태에서 기존의 Resnet은 7.61%의 에러율를 보여준 반면, 새로운 Resnet은 4.92%의 에러율을 보여줬다(CIFAR-10 DATASET으로 테스트).

<img src="https://cdn-images-1.medium.com/max/1600/1*V2FgD6udOE4xJuu_R7L6qA.png" width= '600'><center>
(a) Previous ResNet [2] (7.61%) (b) New ResNet with Identity Mapping [1] (4.92%) for CIFAR-10 Dataset</center><img>

하지만, 왜 이 구조가 shortcut connection 경로를 더 잘 유지할 수 있었을까?(그림에서 살펴보면, 'Relu layer'를 기존의 shortcut connection 경로에서 conv layer 경로로 이동했다)

이 논문에서는 이러한 현상에 대해 설명해 주고 있으며, 심화학습(ablation study)에서는 Identity mapping의 중요성을 잘 보여주고 있다.

이 구조는 Inception-v3보다 더 좋은 결과를 보여주었으며, 2016년 ECCY 논문으로 발표했다.

## What are covered

1. Explanations of the Importance of Identity Mapping
2. Ablation Study
3. Comparison with State-of-the-art Approaches
4. Conclusions

# 1. Explanations of the Importance of Identity Mapping

forward feeding, backpropagation and gradient updates 과정은 딥러닝을 비밀스럽게(어렵게..) 보이도록 한다. 이 논문에서는 이런 과정을 매우 잘 설명하고 있다.

** 용어 설명
* pre-activation : weight layer 이전에 activation function(BN and ReLU)를 쌓는 방법.
* post-activation : weight layer 이후에 activation function(BN and ReLU)를 쌓는 방법.

## 1.1. Feed Forward

Identity Mapping의 Resnet에서는 input과 output 사이 shortcut connection 경로(논문에서는 information path, 맨 위 그림에서 회색 화살표)에 conv나 BN, Relu를 추가하지않고 순수하게 유지하는 것이 핵심이다(essential).

$x_l$ 은 input layer이고, $F$는 conv, BN, Relu를 나타내는 함수(function)면, 아래와 같이 표현할 수 있다.

<img src="https://cdn-images-1.medium.com/max/1600/1*nAhwzaJ0RMGd2nqu50hS3Q.png" width= '300'><center><br>
One Particular Layer</small><img>

<img src="https://cdn-images-1.medium.com/max/1600/1*_57CC9O1VHiZe0DdjLnpdw.png" width= '300'><center><br>L layers from l-th layer
</center><img>

즉, input signal인 $x_l$이 그대로 유지되는 것을 볼 수 있다.

## 1.2. Backpropagation

Backpropagation 과정에서는, 두 부분으로 분해한(decomposed) 형태에서 gradient를 얻을 수 있다. 

<img src="https://cdn-images-1.medium.com/max/1600/1*uHUGqh8iJnH_e2WTHH18eA.png
" width= '600'><br><center>
Gradient which decomposed into two additive terms
</center><img><br>

괄호안을 보면 network에 상관없이 왼쪽에 1인 term을 항상 얻을 수 있다. 그리고 오른쪽 term은 절대 -1이 될 수 없기때문에 gradient값은 0이 될 수 없게된다. 즉 gradient가 절대 사라지지(vanishing) 않는다.

## 1.2. Backpropagation When Identity Mapping Is Violated

#### "On the Importance of Identity Skip Connections"

반대로 왼쪽 term이 1이 아닌 값을 넣어보면 다음과 같다.

<img src="https://cdn-images-1.medium.com/max/1600/1*qXDsS_I-q02zBgJvrTh8Dg.png
" width= '300'><br><center>
    One Particular Layer
</center><img>

<img src="https://cdn-images-1.medium.com/max/1600/1*nrxm2PshU5xnjGAo4b1zDw.png
" width= '300'><br><center>
L layers from l-th layer
</center><img>

<img src="https://cdn-images-1.medium.com/max/1600/1*xw7SoGOQoludcyrQo8c0UQ.png
" width= '300'><br><center>
    Gradient which decomposed into two additive terms
</center><img>

이렇게 되면 왼쪽 term은 $λ$에 따라 변하는 값이 된다.

만약 $λ > 1$ 이면, 왼쪽 term이 점점 커지고, <strong>gradient exploding</strong>이 발생하게 된다. 여기서 하나 기억해야할 점은 gradient가 넘치게(exploded)되면, loss가 수렴하지(converged)않게 된다.

만약 $λ < 1$ 이면, 왼쪽 term이 점점 작아지고, <strong>gradient vanishing</strong>이 발생한다. gradient를 학습할 수 없기때문에 loss function의 높은 지점(plateau)에 머무르게 되고, 끝내는 large loss로 수렴하게 된다.

따라서, input과 output 사이의 shortcut connection를 conv, BN, Relu없이 순수하게 유지해야 한다는 점을 알 수 있다.

# 2. Ablation Study

## 2.1. Various types of shortcut connections

여러 형태의 shortcut connection로 구성한 110-layer Resnet을 'CIFAR-10' dataset으로 테스트를 했다.

<img src="https://cdn-images-1.medium.com/max/2000/1*9lIcsEfLaxvb3hNW-WNIuQ.png
" width= '800'><br><center>
    Performance of Various Types of Shortcut Connections
</center><img>

* Original : 기존 버전의 Resnet 구조, 6.61%의 에러율
* Constant Scaling : $λ=0.5$, gradient vanishing 문제가 발생한 것을 볼 수 있으며, careful selection of bias bg을 이용한 결과 12.35%의 에러율
* Exclusive Gating & Shortcut-only gating : path 값을 1로 유지하면서, shortcut path에 복잡성을 부여한 구조. 두 가지 방법 모두 좋은 결과를 얻지 못하였다. (gating function : $g(x) = σ(Wgx + bg)$,  $σ = sigmoid$)
* 1×1 Conv Shortcut : 기존 Resnet의 C type 구조와 비슷하게 구성한 모델. 기존버전에는 해당 설정을 추가했을 때 더 좋은 결과를 얻었지만, 여기에서는 residual unit이 너무 많아져서(too deep) 좋은 성능을 나타내지 않았다.
* Dropout Shortcut : $λ=0.5$ 인 dropout을 추가.

논문에서 Shortcut connection path는 정보(information)을 전파하는 가장 직접적인 경로이며, Multiplicative manipulations(scaling, gating, 1x1 conv, dropout)가 정보의 전파(information propagation)를 방해하고 optimization 문제를 유발한다고 언급한다.

## 2.2. Various Usages of Activation

아래의 실험은 BN과 Relu의 위치를 바꿔보면서 실험한 결과이다.<br>

<img src="https://cdn-images-1.medium.com/max/2000/1*kxgT5JHFEB31auMdj1iwaA.png
" width= '850'><br><center>
    Performance of Various Usages of Activation
</center><img>

* The Previous ResNet & BN After Addition : identity mapping을 하지않고 shortcut connection에 함수를 더한 경우.
* ReLU Before Addition : Relu를 통과한 residual function은 음수(non-negative)가 나올 수 없어 forward propagated signal가 단순히 증가만 하도록 만든다. 하지만 residual function은 음수도 가지고 있을 때가 더 좋은 성능을 보여준다.
* ReLU-only Pre-Activation : BN없이 Relu를 연결한 구조. BN의 장점을 보여주지 못한다.
* Full Pre-Activation : shortcut connection을 순수하게 유지하고, BN을 지난 후에 Relu를 사용한 구조로 가장 좋은 성능을 보여준다.

## 2.3. Advantages of Pre-activation in Twofold

논문에서는 pre-activation을 사용하면 두 가지 장점이 있다고 한다. Optimization이 쉬워지는 점과(기존 구조에 비해서) 모델의 정규성(regularization of the models)을 향상시킨다고 한다. 

### 2.3.1. Ease of Optimization

기존 Resnet 구조를 사용하게 되면(Baseline) 잘못된 Relu layer의 위치때문에 네트워크를 깊게 쌓게되면(1001) 좋지 않은 결과가 나타난다. <strong>pre-activation</strong>을 사용하면, 110에서 1001로 layer가 깊어질수록 좋은 결과를 항상 얻을 수 있다.<br>

<img src="https://cdn-images-1.medium.com/max/1600/1*tZjqttFpjWzSqf9XZQK-5A.png
" width= '600'><br><center>
    Previous ResNet structure (Baseline) vs Pre-activation Unit
</center><img><br>

추가적으로 학습 속도를 빠르게 만들 수 있다고 한다. 본문 맨 위의 그림을 살펴보면, 기존 모델보다 학습 초기 loss가 줄어드는 속도가 훨씬 높은 점을 확인 할 수 있다. 

### 2.3.2. Reducing Overfitting

Pre-activaion unit은 <strong>Regularization</strong> 기능을 해서 수렴 시에 training loss는 조금 높지만, test error는 더 좋은 값을 얻을 수 있다.

<img src="https://cdn-images-1.medium.com/max/1600/1*Z7Mx6XQ_knGc8eT-G_Zt5g.png
" width= '600'><br><center>
    Training Error vs Iterations
</center><img>

# 3. Comparison with State-of-the-art Approaches

## 3.1 CIFAR-10 & CIFAR-100

CIFAR-10, 기존 ResNet-1202는 7.93%를 달성했지만, 논문에서 제안한 pre-activation unit을 사용한 ResNet-1001(기존 구조보다 200 layer가 적음)는 4.62%의 결과를 보여주었다. 

CIFAR-100, 기존 ResNet-1001 구조는 27.82%, pre-activation을 사용한 ResNet-1001은 22.71%의 결과를 보여주었다.

CIFAR-10과 CIFAR-100 모두에서 pre-activation unit을 더한 ResNet-1001 구조가 ResNet-164보다 좋은 결과를 보여주었다(기존 ResNet에서는 layer가 많은 1001 구조에서 더 큰 에러율을 보이고 있다).

<img src="https://cdn-images-1.medium.com/max/1600/1*SrUvoGCBkYOuUfAJLZxRCw.png
" width= '600'><br><center>
    CIFAR-10 & CIFAR-100 Results
</center><img>

## 3.2 ILSVRC

Scale augmentation만 추가했을 때, 기존 방식의 ResNet-152(the winner of ILSVRC 2015, 5.5%)는 ResNet-200(6.0%)보다 좋은 결과를 보여주고 있다. 이는 network가 깊어지면 ReLU의 잘못된 위치로 인한 문제가 발생하기 때문인다.

pre-activation을 적용한 ResNet-200은 5.3%로 기존 ResNet-200보다 좋은 성능을 나타냈다.

Scale, aspect ratio augmentation을 적용했을 때, ResNet-200 with Pre-Activation(4.8%)로  Inception-v3 by Google (5.6%)보다 좋은 성능을 나타냈다.

현재는 Google이 발표한 Inception-ResNet-v2 with pre-activation unit이 4.9%의 에러율을 달성했으며, 점차 낮아질 것으로 예상된다.

ILSVRC에서 ResNet-200는 8개의 GPU로 3주간 학습을 진행했다.

<img src="https://cdn-images-1.medium.com/max/1600/1*HJiVwHvQYB5lRkdpFDLmNg.png
" width= '600'><br><center>
    ILSVRC Image Classification Results
</center><img>

## 4. Conclusions

이번 논문에서는 깊은 Residual network에서 connection mechanisums에 대한 전파 방법(propagation formulation)에 대해 알아보았다. <strong>Identity shortcut connections과 Identity after-addition acivation</strong> 두 가지 방법은 정보의 전파를 부드럽게(smooth) 만들어 주기 위해 필수적이라고 할 수 있다.